In [ ]:
import pandas as pd
import numpy as np
import json
import os
import requests
from datetime import datetime
import time

In [ ]:
API_KEY = 'VJga0fAn'
blacklist = []
retry_set = set()
STATIONS = pd.read_csv('data/stations.csv')

In [ ]:
STATIONS

## Methods

In [ ]:
def drop_null_vals(df,axis='both',subset=[]):
    '''
    Drops columns with all
    nan values from a given 
    data frame.
    
    Parameters
    ----------
    df : DataFrame
        DataFrame for which
        columns are to be
        dropped.
        
    axis : str
        Drops all rows with
        nan if axis=rows,
        all columns if axis=columns,
        and both if axis=both.
        
    subset : list of str
        For all columns in
        subset, remove the
        NaN rows.
    '''
    assert(isinstance(df,pd.DataFrame))
    assert(isinstance(axis,str))
    assert(isinstance(subset,list))
    assert(isinstance(col,str) for col in subset)
    
    df = df.dropna(subset=subset)
    
    if(axis=='rows'):
        df = df.dropna(how='all',axis=0)
    elif(axis=='columns'):
        df = df.dropna(how='all',axis=1)
    elif(axis=='both'):
        df = df.dropna(how='all',axis=0).dropna(how='all',axis=1)
    
    return df

def getWeatherData(latitude,longitude):
    '''
    Returns temperature 
    and humidity data 
    as per the latitude 
    and longitude entered.
    
    Parameters
    ----------
    latitude : float
        Latitude of region
        for fetching the
        weather data
        
    longitude : float
        Longitude of region
        for fetching weather
        data
    '''
    assert(isinstance(latitude,float))
    assert(isinstance(longitude,float))
    
    station = getNearbyStation(latitude,longitude)
    if(station is None):
        return
    else:
        print("Station found for (%f,%f):\n%s"%(latitude,longitude,station))
    url = createHourlyURL(station,START_DATE,END_DATE)
    weather_data = dict()
    print("URL for getting weather data for (%f,%f):\n%s"%(latitude,longitude,url))
    response = requests.get(url)
    code = response.status_code
    print("Got response, status = %f"%(code))
    try:
        df=pd.DataFrame(response.json()['data'])
        date_list = list(df['time_local'].unique())
#         print(retry_set)
        retry_set.discard((latitude,longitude))
        for date in date_list:
            mean_temp=df[df['time_local']==date]['temperature'].mean()
            mean_humidity=df[df['time_local']==date]['humidity'].mean()
            df_dict = dict()
            df_dict['temperature'] = mean_temp
            df_dict['humidity'] = mean_humidity
            weather_data[date] = df_dict
    except:
        print('No data found for (%f,%f)'%(latitude,longitude))
        print("Retry has to be done for (%f,%f)"%(latitude,longitude))
        coordinates = (latitude,longitude)
        retry_set.add(coordinates)
    
    return weather_data

def getNearbyStation(latitude,longitude):
    '''
    Given the latitude and
    longitude of a area,
    returns the nearest station
    to it.
    
    Parameters
    ----------
    latitude : float
        Latitude of region
        for fetching the
        weather data
        
    longitude : float
        Longitude of region
        for fetching weather
        data
    '''
    assert(isinstance(latitude,float))
    assert(isinstance(longitude,float))

    url = createStationURL(latitude,longitude)

    try:
        filter1 = stations['Latitude']==latitude
        filter2 = stations['Longitude']==longitude
        return stations.where(filter1 & filter2).dropna(how='all')['Weather Station ID'][0]
    except:
        try:
            print('Station does not exist in "Stations.csv", fetching from api...')
            response = requests.get(url)
            code = response.status_code
            result = response.json()['data'][0]['id']
            df = pd.DataFrame({"Weather Station ID":[result], "Latitude":[latitude], "Longitude":[longitude]})
            STATIONS.append(df, ignore_index=True)
            print('Station fetched for (%f,%f) is: %s'%(latitude,longitude,result))
            return result
        except Exception as ex:
            print('No station found for (%f,%f). URL: %s'%(latitude,longitude,url))
            print('exception: ',ex)
            if(code == 403):
                print("Retry has to be done for (%f,%f)"%(latitude,longitude))
                retry_set.add((latitude,longitude))
            return
        
def createStationURL(latitude,longitude):
    '''
    Returns station URL
    for given latitude
    and longitude.
    
    Parameters
    ----------
    latitude : float
        Latitude of region
        for fetching the
        weather data
        
    longitude : float
        Longitude of region
        for fetching weather
        data
    '''
    assert(isinstance(latitude,float))
    assert(isinstance(longitude,float))
    
    return 'https://api.meteostat.net/v1/stations/nearby?lat='+str(latitude)+'&lon='+str(longitude)+'&limit=1&key='+API_KEY

def createHourlyURL(station_id,start_date,end_date):
    '''
    Creates weather URL
    for given station,
    start date and end
    date.
    
    Parameters
    ----------
    station_id : str
        Station id of the
        region for which
        data is to be fetched
        
    start_date : str
        Date starting from which
        data is to be fetched.
        
    end_date : str
        Date ending at which
        data is to be fetched
    '''
    assert(isinstance(station_id,str))
    assert(isinstance(start_date,str))
    assert(isinstance(end_date,str))
    
    url = 'https://api.meteostat.net/v1/history/hourly?station='+station_id+'&start='+start_date+'&end='+end_date+'&time_zone=Europe/London&time_format=Y-m-d&key='+API_KEY

    return url

def getCompleteWeatherData(coordinate_list,complete_weather_data=dict(),flag=True):
    '''
    Returns consolidated weather
    data for all the coordinates
    in list of coordinates.
    
    Parameters
    ----------
    coordinate_list : list(tuple)
        List of coordinates to
        be evaluated.
        
    complete_weather_data : dict
        Map of coordinate to
        weather data.
    
    flag : bool
        If True, means some
       coordinate_listts failed, retry
        the failed requests.
    '''
    assert(isinstance(flag,bool))
    assert(isinstance(complete_weather_data,dict))
    assert(isinstance(coordinate_list,list))
    
    try:
        if(flag==True):
            flag=False
            for lat_long in lat_long_list:
                coordinate = (str(lat_long[0]),str(lat_long[1]))
                if((coordinate not in complete_weather_data) and (coordinate not in blacklist)):
                    flag = True
                    complete_weather_data[(str(lat_long[0]),str(lat_long[1]))] = getWeatherData(lat_long[0],lat_long[1])
                    time.sleep(10)
            return getCompleteWeatherData(coordinate_list,complete_weather_data,flag)
        else:
            return complete_weather_data
    except Exception as ex:
        print(ex)
        return complete_weather_data


## Data

### 1. covid_19_data.csv

Sno - Serial number<br/>
ObservationDate - Date of the observation in MM/DD/YYYY<br/>
Province/State - Province or state of the observation (Could be empty when missing)<br/>
Country/Region - Country of observation<br/>
Last Update - Time in UTC at which the row is updated for the given province or country. (Not standardised and so please clean before using it)<br/>
Confirmed - Cumulative number of confirmed cases till that date<br/>
Deaths - Cumulative number of of deaths till that date<br/>
Recovered - Cumulative number of recovered cases till that date

### 2. COVID_open_line_list_data.csv and COVID19_line_list_data.csv

Individual level data information
<br/><br/>

ID<br/>
age<br/>
sex<br/>
city<br/>
province<br/>
country<br/>
wuhan(0)_not_wuhan(1)<br/>
latitude<br/>
longitude<br/>
geo_resolution<br/>
date_onset_symptoms<br/>
date_admission_hospital<br/>
date_confirmation<br/>
symptoms<br/>
lives_in_Wuhan<br/>
travel_history_dates<br/>
travel_history_location<br/>
reported_market_exposure<br/>
additional_information<br/>
chronic_disease_binary<br/>
chronic_disease<br/>
source<br/>
sequence_available<br/>
outcome<br/>
date_death_or_discharge<br/>
notes_for_discussion<br/>
location<br/>
admin3<br/>
admin2<br/>
admin1<br/>
country_new<br/>
admin_id<br/>
data_moderator_initials<br/>

### Region wise data

In [ ]:
region_wise_data = drop_null_vals(pd.read_csv('data/covid_19_data.csv'),axis="both")
region_wise_data

### Individual information

In [ ]:
open_line_list = drop_null_vals(pd.read_csv('data/COVID19_open_line_list.csv'),axis='both')
open_line_list

In [ ]:
line_list = drop_null_vals(pd.read_csv('data/COVID19_line_list_data.csv'),'both')
line_list

### Time series data (John Hopkins)

In [ ]:
try:
    confirmed_url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv'
    deaths_url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Deaths.csv'
    recovered_url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Recovered.csv'
    time_series_confirmed = drop_null_vals(pd.read_csv(confirmed_url,error_bad_lines=False))
    time_series_confirmed.to_csv('data/time_series_covid_19_confirmed.csv')
    
    time_series_deaths = drop_null_vals(pd.read_csv(deaths_url,error_bad_lines=False))
    time_series_deaths.to_csv('data/time_series_covid_19_deaths.csv')
    
    time_series_recovered = drop_null_vals(pd.read_csv(recovered_url,error_bad_lines=False))
    time_series_recovered.to_csv('data/time_series_covid_19_recovered.csv')
except:
    # data not able to be fetched from git, fetching from local system
    print("Data not able to fetch from git. Using local filesystem.")
    time_series_confirmed = drop_null_vals(pd.read_csv('data/time_series_covid_19_confirmed.csv'))
    time_series_deaths = drop_null_vals(pd.read_csv('data/time_series_covid_19_deaths.csv'))
    time_series_recovered = drop_null_vals(pd.read_csv('data/time_series_covid_19_recovered.csv'))

In [ ]:
# Dropping today's data
time_series_confirmed.drop(time_series_confirmed.columns[len(time_series_confirmed.columns)-1], axis=1, inplace=True)
time_series_deaths.drop(time_series_deaths.columns[len(time_series_deaths.columns)-1], axis=1, inplace=True)
time_series_recovered.drop(time_series_recovered.columns[len(time_series_recovered.columns)-1], axis=1, inplace=True)

In [ ]:
series_confirmed_coord = drop_null_vals(time_series_confirmed,subset=['Lat','Long'])
series_confirmed_coord

### Temperature and humidity data

In [ ]:
lat_long_list = list(series_confirmed_coord[['Lat','Long']].values)

<b><u>API Sample Calls</u></b>

API to fetch station: https://api.meteostat.net/v1/stations/nearby?lat=1.283&lon=103.83&limit=1&key=XXXXXXX<br/>
API to fetch daily historical data(not used any more): https://api.meteostat.net/v1/history/daily?station=10637&start=2017-01-01&end=2017-12-31&key=XXXXXXXX<br/>
API to fetch historical hourly data: https://api.meteostat.net/v1/history/hourly?station=03772&start=2019-05-02&end=2019-05-11&time_zone=Europe/London&time_format=Y-m-d%20H:i&key=XXXXXXXX


In [ ]:
START_DATE = str(datetime.strptime(series_confirmed_coord.columns[4], '%m/%d/%y').date())
END_DATE = str(datetime.strptime(series_confirmed_coord.columns[-1], '%m/%d/%y').date())
complete_weather_data = dict()

In [ ]:
for lat_long in lat_long_list:
    if(str(lat_long[0])+str(lat_long[1]) not in complete_weather_data):
        complete_weather_data[str(lat_long[0])+str(lat_long[1])] = getWeatherData(lat_long[0],lat_long[1])
        time.sleep(60)

complete_weather_data

In [ ]:
temp = getCompleteWeatherData(lat_long_list)
STATIONS.to_csv(r'data\stations.csv', index=False)

In [ ]:
pd.DataFrame(temp).dropna(how="all",axis=1)

In [ ]:
new_dict = {}
for key in list(temp.keys()):
    new_dict[str(key[0])+","+str(key[1])] = temp[key]
    
my_dict = pd.DataFrame(new_dict).dropna(how="all",axis=1).to_dict()
with open('data/weather_mapping_data.json', 'w') as json_file:
    json.dump(my_dict, json_file)

In [ ]:
last_column = time_series_confirmed.columns[len(time_series_confirmed.columns)-1]
sorted_series=time_series_confirmed.sort_values(by=last_column,ascending=False)
sorted_series = sorted_series[sorted_series[last_column]>1000]

In [ ]:
sorted_series